## Initial Dev

In [1]:
import datajoint as dj
dj.conn()
import csv

Connecting root@localhost:3306


In [2]:
from workflow_session.pipeline import lab, subject, session
from workflow_session.ingest import *

In [3]:
ingest_lab(); ingest_subjects();ingest_sessions()


---- Insert 2 entry(s) into lab tables ----

---- Insert 2 entry(s) into project table ----

---- Insert entry(s) into publication/keyword tables ----

---- Insert 2 entry(s) into protocol tables ----

---- Insert 2 entry(s) into subject tables ----

---- Insert 2 entry(s) into session.Session ----


In [4]:
print(lab.Lab & 'lab="LabA"')
print(lab.Project & 'project="ProjA"')
print(lab.Protocol() & 'protocol="ProtA"')

lab_info = (lab.Lab & 'lab="LabA"').fetch1()
proj_info = (lab.Project & 'project="ProjA"').fetch1()
prot_info = (lab.Protocol() & 'protocol="ProtA"').fetch1()

*lab     lab_name       institution    address        time_zone    
+------+ +------------+ +------------+ +------------+ +-----------+
LabA     The Example La Example Uni    221B Baker St, UTC+0        
 (Total: 1)

*project    project_descri repositoryurl  repositoryname pharmacology   viruses     slices     stimulus       surgery    
+---------+ +------------+ +------------+ +------------+ +------------+ +---------+ +--------+ +------------+ +---------+
ProjA       Example projec https://github element-lab                                          videos generat            
 (Total: 1)

*protocol    protocol_type  protocol_descr
+----------+ +------------+ +------------+
ProtA        IRB expedited  Protocol for m
 (Total: 1)



In [5]:
import pynwb
from element_session.export import *
session_key=(session.Session&'subject="subject5"').fetch1('KEY')
mylab_key = (lab.Lab & 'lab="LabA"').fetch1('KEY')
myproj_key= (lab.Project & 'project="ProjA"').fetch1('KEY')
myprot_key= (lab.Protocol() & 'protocol="ProtA"').fetch1('KEY')
session_to_nwb(session_key,lab_key=mylab_key,project_key=myproj_key)

/Users/cb/miniconda3/envs/venv-nwb/lib/python3.8/site-packages/pynwb/file.py:753: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")


root pynwb.file.NWBFile at 0x140690194467040
Fields:
  file_create_date: [datetime.datetime(2021, 12, 1, 14, 22, 4, 992693, tzinfo=tzlocal())]
  identifier: subject5_20200415_111638
  institution: Example Uni
  session_description: Successful data collection, no notes
  session_start_time: 2020-04-15 11:16:38-05:00
  timestamps_reference_time: 2020-04-15 11:16:38-05:00

In [32]:
mynwbfile

root pynwb.file.NWBFile at 0x140337549119696
Fields:
  file_create_date: [datetime.datetime(2021, 12, 1, 14, 21, 26, 638467, tzinfo=tzlocal())]
  identifier: subject5_20200415_111638
  institution: Example Uni
  protocol: ProtA
  session_description: Successful data collection, no notes
  session_start_time: 2020-04-15 11:16:38-05:00
  timestamps_reference_time: 2020-04-15 11:16:38-05:00

In [100]:
session_key=(session.Session&'subject="subject5"').fetch1('KEY')
session_identifier = {}
for k, v in session_key.items():
    session_identifier[k] = v.strftime('%Y%m%d_%H%M%S') if isinstance(v, datetime) else v

session_info = (session.Session & session_key).join(session.SessionNote, left=True).fetch1()

def mytuple():
    identifier='_'.join(session_identifier.values()),
    session_description=session_info['session_note'] if session_info['session_note'] else '',
    session_start_time=session_info['session_datetime']
    return identifier,session_description,session_start_time
info=dict(identifier='_'.join(session_identifier.values()),
    session_description='Note',
    session_start_time=session_info['session_datetime'],
    institution='')
info={k: v for k, v in info.items() if v} #drop empty
asstring = ','.join('='.join((str(key),val)) for (key,val) in info.items())

In [12]:
a=None;b=None;c=None
if [x for x in (a, b,c) if x is not None]: print('g')

## Orig ingest, before generalize

In [29]:
def ingest_lab(lab_csv_path='./user_data/lab/labs.csv',
                project_csv_path='./user_data/lab/projects.csv',
                pubs_csv_path='./user_data/lab/publications.csv',
                keyw_csv_path='./user_data/lab/keywords.csv',
                protocol_csv_path='./user_data/lab/protocols.csv',
                skip_duplicates=True, ignore_extra_fields=True):
        # -------------- Insert new "Lab" --------------
    with open(lab_csv_path, newline= '') as f:
        input_labs = list(csv.DictReader(f, delimiter=','))

    prev_len = len(lab.Lab())
    lab.Lab.insert(input_labs, skip_duplicates=skip_duplicates,
                               ignore_extra_fields=ignore_extra_fields)
    lab.Location.insert(input_labs, skip_duplicates=skip_duplicates,
                               ignore_extra_fields=ignore_extra_fields)
    insert_len = len(lab.Lab()) - prev_len
    
    print(f'\n---- Inserting {insert_len} entry(s) into lab tables ----')

    # -------------- Insert new "Project" --------------
    with open(project_csv_path, newline= '') as f:
        input_projs = list(csv.DictReader(f, delimiter=','))
    
    prev_len = len(lab.Project())
    lab.Project.insert(input_projs, skip_duplicates=skip_duplicates,
                                    ignore_extra_fields=ignore_extra_fields)
    insert_len = len(lab.Project()) - prev_len
    print(f'\n---- Inserting {insert_len} entry(s) into project table ----')

    # -------------- Insert publications + keywords --------------
    with open(pubs_csv_path, newline= '') as f:
        input_pubs = list(csv.DictReader(f, delimiter=','))
    with open(keyw_csv_path, newline= '') as f:
        input_keyw = list(csv.DictReader(f, delimiter=','))
    
    prev_len = len(lab.Project.Publication()) + len(lab.Project.Keywords())
    lab.Project.Publication.insert(input_pubs, skip_duplicates=skip_duplicates,
                                               ignore_extra_fields=ignore_extra_fields)
    lab.Project.Keywords.insert(input_keyw, skip_duplicates=skip_duplicates,
                                            ignore_extra_fields=ignore_extra_fields)
    insert_len = len(lab.Project.Publication()) + len(lab.Project.Keywords()) - prev_len
    
    print(f'\n---- Inserting {insert_len} entry(s) into publication/keyword tables ----')

    # -------------- Insert new "Protocol" --------------
    with open(protocol_csv_path, newline= '') as f:
        input_prots = list(csv.DictReader(f, delimiter=','))
    prev_len = len(lab.Protocol())
    lab.Protocol.insert(input_prots, skip_duplicates=skip_duplicates,
                                    ignore_extra_fields=ignore_extra_fields)
    lab.ProtocolType.insert(input_prots, skip_duplicates=skip_duplicates,
                                    ignore_extra_fields=ignore_extra_fields)
    insert_len = len(lab.Protocol()) - prev_len
    print(f'\n---- Inserting {insert_len} entry(s) into protocol tables ----')

def ingest_subjects(subject_csv_path='./user_data/animal/subjects.csv'):
    # -------------- Insert new "Subject" --------------
    with open(subject_csv_path, newline= '') as f:
        input_subjects = list(csv.DictReader(f, delimiter=','))
    print(f'\n---- Inserting {insert_len} entry(s) into subject tables ----')
    subject.Subject.insert(input_subjects, skip_duplicates=skip_duplicates,
                                    ignore_extra_fields=ignore_extra_fields)
    subject.SubjectDeath.insert(input_subjects, skip_duplicates=skip_duplicates,
                                    ignore_extra_fields=ignore_extra_fields)
    subject.SubjectCullMethod.insert(input_subjects, skip_duplicates=skip_duplicates,
                                    ignore_extra_fields=ignore_extra_fields)
    ## Skipped allele info

def ingest_sessions(session_csv_path='./user_data/session/sessions.csv'):
    with open(session_csv_path, newline= '') as f:
        input_sessions = list(csv.DictReader(f, delimiter=','))

    print(f'\n---- Inserting {insert_len} entry(s) into session.Session ----')
    session.Session.insert(input_sessions, skip_duplicates=skip_duplicates,
                                    ignore_extra_fields=ignore_extra_fields)
    session.SessionDirectory.insert(input_sessions, skip_duplicates=skip_duplicates,
                                    ignore_extra_fields=ignore_extra_fields)

    ## Type Error: 'NoneType' object is not iterable
    # session.SessionNote.insert(input_sessions, skip_duplicates=skip_duplicates,
                                    # ignore_extra_fields=ignore_extra_fields)


if __name__ == '__main__':
    ingest_lab()
    ingest_subjects()
    ingest_sessions()


---- Inserting 0 entry(s) into lab tables ----

---- Inserting 0 entry(s) into project table ----

---- Inserting 0 entry(s) into publication/keyword tables ----

---- Inserting 0 entry(s) into protocol tables ----

---- Inserting 0 entry(s) into subject tables ----

---- Inserting 0 entry(s) into session.Session ----


In [1]:
pwd

'/Volumes/GoogleDrive/My Drive/NWB/workflow-session'

In [1]:
import datajoint as dj; dj.conn()
import pathlib
import csv
import re
from workflow_session.pipeline import lab, subject, session
from workflow_session.paths import get_root_data_dir
import element_data_loader.utils

Connecting root@localhost:3306


In [2]:
from workflow_session.ingest import *
ingest_lab(); ingest_subjects();ingest_sessions()


---- Inserting 0 entry(s) into #location ----


AttributeError: 'dict' object has no attribute 'insert'

In [9]:
table=[lab.Lab()]
for t in table:
    t.describe()

lab                  : varchar(24)                  # Abbreviated lab name
---
lab_name             : varchar(255)                 # full lab name
institution          : varchar(255)                 
address              : varchar(255)                 
time_zone            : varchar(32)                  # UTC offset suggested e.g., UTC+1



In [6]:
if not isinstance(lab.Lab(),list): print([lab.Lab()])

[*lab     lab_name       institution    address        time_zone    
+------+ +------------+ +------------+ +------------+ +-----------+
LabA     The Example La Example Uni    221B Baker St, UTC+0        
LabB     The Other Lab  Other Uni      Oxford OX1 2JD UTC+0        
 (Total: 2)
]


In [5]:
type(lab.Lab())

element_lab.lab.Lab